# Chatbot Natura - Documentação

## Visão Geral
O Chatbot Natura é uma aplicação web interativa desenvolvida em Python usando Flask, que simula um assistente virtual para a vendedora de produtos Natura. O chatbot oferece uma interface amigável e moderna para interação com os clientes.

## Tecnologias Utilizadas
- Python
- Flask (Framework web)
- SQLite (Banco de dados)
- HTML/CSS/JavaScript
- Jupyter Notebook

## Funcionalidades Principais

### 1. Interface do Usuário
- Design moderno e responsivo
- Botão flutuante para abrir/fechar o chat
- Interface de chat com mensagens em formato de bolhas
- Menu de opções com botões interativos
- Suporte a emojis e formatação HTML

### 2. Sistema de Cadastro
- Coleta de informações do usuário:
  - Número de telefone (com validação)
  - Nome completo
  - Endereço completo (rua, número, bairro)
- Validação de dados em tempo real
- Persistência de dados no banco SQLite

### 3. Menu de Opções
O chatbot oferece as seguintes opções principais:
- Catálogo online
- Promoções
- Consulta de pedidos
- Informações sobre entrega
- Falar com vendedora
- Encerrar chat

### 4. Gerenciamento de Pedidos
- Visualização do status do pedido
- Detalhes do pedido incluindo:
  - Número do pedido
  - Status atual
  - Tempo estimado de entrega
  - Lista de itens
  - Valores (pedido, entrega e total)
  - Endereço de entrega

### 5. Histórico de Conversas
- Armazenamento de todo o histórico de conversas
- Recuperação do histórico ao retornar ao chat
- Timestamp para cada mensagem

### 6. Integração com WhatsApp
- Link direto para WhatsApp da vendedora
- Mensagem pré-formatada para iniciar conversa

## Como Usar

1. **Iniciar o Chatbot**
   - Execute o notebook `app_natura.ipynb`
   - Clique no link fornecido para abrir a interface web

2. **Primeiro Acesso**
   - Clique no botão flutuante para abrir o chat
   - Forneça seu número de telefone
   - Complete o cadastro com suas informações

3. **Interação**
   - Use os botões do menu para navegar
   - Digite mensagens diretamente no campo de texto
   - Pressione Enter ou clique em Enviar para enviar mensagens

4. **Encerramento**
   - Use a opção "Encerrar Chat" para fechar a sessão
   - O histórico será mantido para futuras interações

## Banco de Dados
O sistema utiliza três tabelas principais:
1. `chat`: Armazena perguntas e respostas padrão
2. `users`: Armazena dados dos usuários
3. `chat_history`: Registra o histórico de conversas

## Recursos de Segurança
- Validação de dados de entrada
- Tratamento de erros
- Proteção contra SQL injection
- Validação de formato de telefone e endereço

## Personalização
O chatbot pode ser facilmente personalizado através de:
- Modificação das respostas padrão
- Alteração do estilo visual (CSS)
- Adição de novas funcionalidades
- Atualização do menu de opções

## Limitações
- Requer conexão com internet
- Depende do navegador web
- Necessita de Python e dependências instaladas
- Limitado às funcionalidades pré-programadas


In [ ]:
import os
import sqlite3
from flask import Flask, jsonify, request, render_template_string
from threading import Thread
from werkzeug.serving import make_server
from IPython.display import display, HTML
from datetime import datetime

app = Flask(__name__)

BASE_DIR = os.getcwd()
DB_FILE = os.path.join(BASE_DIR, "chatbot_natura.db")

def init_db():
    if os.path.exists(DB_FILE):
        os.remove(DB_FILE)

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS chat (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            pergunta TEXT NOT NULL UNIQUE,
            resposta TEXT NOT NULL
        )
    ''')

    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            phone TEXT UNIQUE NOT NULL,
            name TEXT,
            street TEXT,
            number TEXT,
            neighborhood TEXT
        )
    ''')

    c.execute('''
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_phone TEXT NOT NULL,
            message_text TEXT NOT NULL,
            sender TEXT NOT NULL,
            timestamp TEXT NOT NULL,
            FOREIGN KEY (user_phone) REFERENCES users(phone)
        )
    ''')
    
    exemplos = [
        ("olá", "Olá! Como posso ajudar você hoje? 😊"),
        ("catálogo online", "Confira nosso catálogo completo em: <a href='' target='_blank' style='color: #A020F0; text-decoration: underline; font-weight: bold;'>Clique aqui!</a>"),
        ("promoção", "Na compra de 3 produtos, o frete é grátis!"),
        ("pedido", "Certo! Vamos verificar o status do seu pedido. Por favor, aguarde um momento..."),
        ("entrega", "Entregamos em até 5 dias úteis após a confirmação do pedido. calcule seu frete em: <a href='https://www.correios.com.br/a-a-z/c/frete-correios' target='_blank' style='color: #A020F0; text-decoration: underline; font-weight: bold;'>Clique aqui!</a>."),
        ("falar com a vendedora", "Para falar com a vendedora, clique no link do WhatsApp. 😊"),
        ("encerrar chat", "Foi um prazer ajudar! Se precisar de algo mais, é só abrir o chat novamente. 😊"),
    ]

    for pergunta, resposta in exemplos:
        c.execute("INSERT OR IGNORE INTO chat (pergunta, resposta) VALUES (?, ?)", (pergunta, resposta))
            
    conn.commit()
    conn.close()

init_db()

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="pt-BR">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1" />
<title>Natura - Chatbot</title>
<style>
    body { font-family: 'Segoe UI', sans-serif; background: #F5EEFD; margin: 0; }
    header, footer { background-color: #9370DB; color: white; text-align: center; padding: 20px; }
    main { padding: 40px; max-width: 800px; margin: auto; }
    h1 { color: #A020F0; }
    #chatbot-toggle {
        position: fixed;
        bottom: 25px;
        right: 25px;
        width: 60px; height: 60px;
        border-radius: 50%;
        background: #9370DB;
        display: flex; justify-content: center; align-items: center;
        cursor: pointer;
        z-index: 999;
        box-shadow: 0 6px 12px rgba(147, 112, 219, 0.4); /* Sombra mais destacada e colorida */
    }
    #chatbot-toggle svg { width: 30px; height: 30px; fill: white; }

    #chatbot-box {
        position: fixed;
        bottom: 100px;
        right: 25px;
        width: 350px;
        max-height: 500px;
        background: white;
        border-radius: 20px; /* Mais arredondado */
        box-shadow: 0 10px 25px rgba(0,0,0,0.15); /* Sombra mais suave e espalhada */
        display: none;
        flex-direction: column;
        overflow: hidden;
        z-index: 1000;
    }
    #chatbot-header {
        background: #9370DB;
        color: white;
        padding: 15px; /* Maior padding */
        font-weight: bold;
        display: flex;
        justify-content: space-between;
        border-top-left-radius: 20px;
        border-top-right-radius: 20px;
    }
    #chat-log {
        padding: 10px;
        overflow-y: auto;
        flex-grow: 1;
        background: #F8F0FF; /* Fundo mais claro e suave */
        display: flex;
        flex-direction: column;
        gap: 10px;
    }
    .message {
        max-width: 80%;
        padding: 12px; /* Maior padding */
        border-radius: 16px; /* Mais arredondado */
        font-size: 15px; /* Tamanho de fonte ligeiramente maior */
    }
    .user { align-self: flex-end; background: #DDA0DD; color: #3A3A3A; } /* Tom de roxo mais suave e quente */
    .bot { align-self: flex-start; background: #E6E6FA; color: #3A3A3A; } /* Lavanda claro */
    
    .bot a {
        color: #A020F0;
        text-decoration: underline;
        font-weight: bold;
    }
    .bot a:hover {
        color: #8A00C2;
    }

    .menu-message {
        align-self: flex-start;
        background: #E6E6FA;
        border-radius: 16px;
        padding: 12px;
        max-width: 80%;
        font-size: 15px;
        display: flex;
        flex-direction: column;
        gap: 8px; /* Maior espaçamento entre botões */
    }

    .menu-message .menu-title {
        margin-bottom: 5px;
        font-weight: bold;
        color: #6A0DAD; /* Tom mais escuro para o título do menu */
    }

    .menu-message button {
        margin: 0;
        padding: 10px 15px; /* Maior padding */
        border-radius: 20px; /* Mais arredondado */
        border: none;
        background: #9370DB;
        color: white;
        cursor: pointer;
        font-size: 15px;
        text-align: center;
        width: 100%;
        box-sizing: border-box;
        box-shadow: 0 4px 8px rgba(147, 112, 219, 0.3); /* Sombra mais sutil nos botões */
    }
    .menu-message button:hover {
        background: #8A2BE2; /* Um azul violeta para hover */
    }

    .consolidated-order-status {
        align-self: flex-start;
        background: #E6E6FA;
        border-radius: 16px;
        padding: 12px;
        max-width: 80%;
        font-size: 15px;
        margin-bottom: 5px;
        color: #3A3A3A;
    }
    .consolidated-order-status strong {
        color: #A020F0;
    }
    .consolidated-order-status ul {
        list-style: none;
        padding: 0;
        margin-top: 8px; /* Mais espaçamento */
    }
    .consolidated-order-status li {
        margin-bottom: 5px; /* Mais espaçamento */
    }


    #input-area {
        display: flex;
        padding: 10px;
        border-top: 1px solid #E0E0E0; /* Borda mais clara */
        background: #fff;
        border-bottom-left-radius: 20px;
        border-bottom-right-radius: 20px;
    }
    #user-input {
        flex-grow: 1;
        padding: 10px; /* Maior padding */
        border: 1px solid #D8BFD8; /* Borda em tom suave de roxo */
        border-radius: 25px; /* Mais arredondado */
        font-size: 15px;
    }
    .send-btn {
        margin-left: 10px;
        padding: 10px 18px; /* Maior padding */
        background: #9370DB;
        color: white;
        border: none;
        border-radius: 25px; /* Mais arredondado */
        cursor: pointer;
        box-shadow: 0 4px 8px rgba(147, 112, 219, 0.3);
    }
    .send-btn:hover {
        background: #8A2BE2;
    }
</style>
</head>
<body>

<header>Natura - Chatbot</header>
<main>
    <h1>Bem-vindo(a) à Natura!</h1>
    <p>Use o chatbot no canto inferior direito para tirar suas dúvidas sobre nosso catálogo, promoções e muito mais!</p>
</main>
<footer>&copy; 2025 Natura </footer>

<div id="chatbot-toggle" onclick="toggleChatbot()">
    <svg width="30" height="30" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg">
        <path d="M12 2C8.686 2 6 4.686 6 8C6 11.314 12 22 12 22C12 22 18 11.314 18 8C18 4.686 15.314 2 12 2ZM12 11C10.343 11 9 9.657 9 8C9 6.343 10.343 5 12 5C13.657 5 15 6.343 15 8C15 9.657 13.657 11 12 11Z" fill="white"/>
        <path d="M12 9.5C11.1716 9.5 10.5 = 8.82843 10.5 8C10.5 7.17157 11.1716 6.5 12 6.5C12.8284 6.5 13.5 7.17157 13.5 8C13.5 8.82843 12.8284 9.5 12 9.5Z" fill="#F0F0F0"/>
    </svg>
</div>

<div id="chatbot-box">
    <div id="chatbot-header">
        Chatbot 🌸
        <button onclick="toggleChatbot()">×</button>
    </div>
    <div id="chat-log"></div>
    <div id="input-area">
        <input type="text" id="user-input" placeholder="Digite sua mensagem..." />
        <button class="send-btn" onclick="sendMessage()">Enviar</button>
    </div>
</div>

<script>
    const chatbotBox = document.getElementById('chatbot-box');
    const chatLog = document.getElementById('chat-log');
    const userInput = document.getElementById('user-input');
    const sendBtn = document.querySelector('.send-btn');

    let hasGreeted = false;
    let collectingInfo = false;
    let currentInfoStep = 0;
    let customerName = '';
    let customerPhone = '';
    let customerStreet = '';
    let customerNumber = '';
    let customerNeighborhood = '';
    let customerAddress = '';

    const menuOptions = [
        { text: 'Catálogo online', value: 'catálogo online' },
        { text: 'Promoção', value: 'promoção' },
        { text: 'Pedido', value: 'pedido' },
        { text: 'Entrega', value: 'entrega' },
        { text: 'Falar com a vendedora', value: 'falar com a vendedora' },
        { text: 'Encerrar Chat', value: 'encerrar chat', isSpecial: true }
    ];

    function appendMessage(text, sender) {
        const div = document.createElement('div');
        div.className = 'message ' + sender;
        if (sender === 'bot') {
            div.innerHTML = text;
        } else {
            div.innerText = text;
        }
        chatLog.appendChild(div);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    function appendMenuMessage(title, options) {
        const menuDiv = document.createElement('div');
        menuDiv.className = 'menu-message bot';

        const titleSpan = document.createElement('span');
        titleSpan.className = 'menu-title';
        titleSpan.innerText = title;
        menuDiv.appendChild(titleSpan);

        options.forEach(option => {
            const button = document.createElement('button');
            button.innerText = option.text;
            button.onclick = () => handleMenuButtonClick(option.value, option.isSpecial);
            menuDiv.appendChild(button);
        });

        chatLog.appendChild(menuDiv);
        chatLog.scrollTop = chatLog.scrollHeight;
    }

    function setInputActive(active) {
        userInput.disabled = !active;
        sendBtn.disabled = !active;
    }

    async function loadChatHistory(phone) {
        try {
            const response = await fetch('/get_chat_history', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ phone: phone })
            });
            const data = await response.json();
            if (data.history && data.history.length > 0) {
                chatLog.innerHTML = '';
                data.history.forEach(msg => {
                    appendMessage(msg.message_text, msg.sender);
                });
                appendMessage('Bem-vindo(a) de volta! O que podemos fazer por você hoje?', 'bot');
            } else {
                appendMessage('Bem-vindo(a)! Parece que é sua primeira vez aqui. Como posso ajudar você hoje?', 'bot');
            }
        } catch (error) {
            console.error('Erro ao carregar histórico de chat:', error);
            appendMessage('Desculpe, não consegui carregar seu histórico de chat. Como posso ajudar você?', 'bot');
        }
    }

    async function saveMessageToHistory(userPhone, messageText, sender) {
        try {
            await fetch('/save_chat_message', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    phone: userPhone,
                    message_text: messageText,
                    sender: sender
                })
            });
        } catch (error) {
            console.error('Erro ao salvar mensagem no histórico:', error);
        }
    }

    function startCollectingInfo() {
        collectingInfo = true;
        currentInfoStep = 0;
        setInputActive(true);
        appendMessage('Olá! Para começarmos, por favor, digite seu número de telefone (apenas números, com DDD).', 'bot');
        userInput.focus();
    }

    async function processCollectedInfo(msg) {
        if (currentInfoStep === 0) {
            if (!/^\d{10,11}$/.test(msg)) {
                appendMessage('Por favor, digite um número de telefone válido (apenas números, com DDD).', 'bot');
            } else {
                customerPhone = msg;
                setInputActive(false);
                appendMessage('Verificando suas informações...', 'bot');

                try {
                    const response = await fetch('/get_user_data', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ phone: customerPhone })
                    });
                    const userData = await response.json();

                    if (userData.found) {
                        customerName = userData.name;
                        customerStreet = userData.street || '';
                        customerNumber = userData.number || '';
                        customerNeighborhood = userData.neighborhood || '';
                        customerAddress = `Rua: ${customerStreet}, Nº: ${customerNumber}, Bairro: ${customerNeighborhood}`;

                        collectingInfo = false;
                        currentInfoStep = 0;
                        appendMessage(`Bem-vindo(a) de volta, ${customerName}!`, 'bot');
                        await loadChatHistory(customerPhone);
                        appendMenuMessage('O que podemos fazer por você hoje?', menuOptions);
                    } else {
                        appendMessage('Número não encontrado. Vamos fazer seu cadastro!', 'bot');
                        currentInfoStep = 1;
                        appendMessage('Qual é o seu nome completo?', 'bot');
                    }
                } catch (error) {
                    console.error('Erro ao verificar telefone:', error);
                    appendMessage('Desculpe, houve um erro ao verificar seu telefone. Por favor, tente novamente.', 'bot');
                } finally {
                    setInputActive(true);
                    userInput.focus();
                }
            }
        } else if (currentInfoStep === 1) {
            const nameRegex = /^[A-Za-z\u00C0-\u00FF\s]+$/;
            if (!nameRegex.test(msg)) {
                appendMessage('Por favor, digite um nome válido, usando apenas letras e espaços.', 'bot');
            } else if (msg.length < 2) {
                appendMessage('Por favor, digite um nome mais completo.', 'bot');
            } else {
                customerName = msg;
                appendMessage(`Ok, ${customerName}!`, 'bot'); 
                currentInfoStep = 2;
                appendMessage('Por favor, digite o nome da sua rua.', 'bot');
            }
        } else if (currentInfoStep === 2) {
            if (msg.length < 3) {
                appendMessage('Por favor, digite um nome de rua válido e mais completo.', 'bot');
            } else {
                customerStreet = msg;
                appendMessage('Rua registrada.', 'bot');
                currentInfoStep = 3;
                appendMessage('Qual é o número da sua casa ou apartamento? Se for apartamento, inclua o número dele (Ex: 123 ou 123, apto 401).', 'bot');
            }
        } else if (currentInfoStep === 3) {
            const numberRegex = /^(\d+)(\s*,\s*apto\s*\d+)?$/i;
            if (!numberRegex.test(msg)) {
                appendMessage('Por favor, digite um número válido para sua casa ou apartamento (Ex: 123 ou 123, apto 401).', 'bot');
            } else {
                customerNumber = msg;
                appendMessage('Número registrado.', 'bot');
                currentInfoStep = 4;
                appendMessage('Qual é o seu bairro?', 'bot');
            }
        } else if (currentInfoStep === 4) {
            if (msg.length < 3) {
                appendMessage('Por favor, digite um nome de bairro válido e mais completo.', 'bot');
            } else {
                customerNeighborhood = msg;
                customerAddress = `Rua: ${customerStreet}, Nº: ${customerNumber}, Bairro: ${customerNeighborhood}`;
                appendMessage('Bairro registrado.', 'bot');
                
                try {
                    await fetch('/save_user_data', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({
                            phone: customerPhone,
                            name: customerName,
                            street: customerStreet,
                            number: customerNumber,
                            neighborhood: customerNeighborhood
                        })
                    });
                    appendMessage('Seu cadastro foi concluído com sucesso!', 'bot');
                } catch (error) {
                    console.error('Erro ao salvar dados do usuário:', error);
                    appendMessage('Desculpe, houve um erro ao salvar seus dados. Por favor, tente novamente.', 'bot');
                }

                collectingInfo = false;
                currentInfoStep = 0;
                appendMessage('Agora, como posso ajudar você?', 'bot');
                appendMenuMessage('Selecione uma opção:', menuOptions);
            }
        }
        userInput.focus();
    }

    function showConsolidatedOrderStatus() {
        const customerOrder = {
            id: 'ABCDE-12345',
            status: 'Pedido sendo preparado',
            estimatedTime: '30-55 min',
            items: ['Perfume Essencial Oud (100ml) .......R$ 150,00', 'Lápis de olho Faces (preto) .......R$ 25,00', 'Batom Power Stay (vermelho) .......R$ 40,00'],
            deliveryAddress: customerAddress || 'Endereço não informado',
        };

        appendMessage('Buscando informações do seu pedido...', 'bot');

        setTimeout(() => {
            if (customerOrder) {
                let itemsList = customerOrder.items.map(item => `<li>${item}</li>`).join('');

                let orderHtml = `<div class="consolidated-order-status bot">
                    <strong>Status do Pedido #${customerOrder.id}</strong><br>
                    <ul>
                        <li>Status Atual: <strong>${customerOrder.status}</strong></li>
                        <li>Tempo estimado de entrega: ${customerOrder.estimatedTime}</li>
                        <li>Itens:</li>
                        <ul>
                            ${itemsList}
                        </ul>
                        <li>Valor do pedido: R$ 215,00 </li>
                        <li>Valor da entrega: R$ 15,00 </li><br>
                        <li>Valor Total: R$ 230,00 </li><br>
                        <li>Endereço de Entrega: ${customerOrder.deliveryAddress}</li>
                        </ul>
                    Agradecemos a sua paciência!
                </div>`;
                chatLog.innerHTML += orderHtml;
                chatLog.scrollTop = chatLog.scrollHeight;
                appendMenuMessage('Selecione outra opção:', menuOptions);
            } else {
                appendMessage('Não encontramos nenhum pedido ativo associado ao seu ID de cliente.', 'bot');
                appendMenuMessage('Selecione outra opção:', menuOptions);
            }
            userInput.focus();
        }, 1500);
    }

    function handleMenuButtonClick(value, isSpecial) {
        setInputActive(false);

        if (customerPhone) {
            saveMessageToHistory(customerPhone, menuOptions.find(opt => opt.value === value)?.text || value, 'user');
        }
        appendMessage(menuOptions.find(opt => opt.value === value)?.text || value, 'user');

        if (value === 'falar com a vendedora') {
            const whatsappNumber = '5511999999999';
            const whatsappLink = `https://wa.me/${whatsappNumber}?text=Olá,%20gostaria%20de%20falar%20com%20a%20vendedora%20da%20Natura.`;
            const botResponse = `Certo! Para falar com a vendedora, clique no link a seguir: <a href="${whatsappLink}" target="_blank" style="color: #A020F0; text-decoration: underline; font-weight: bold;">Falar com a Vendedora no WhatsApp</a>`;
            appendMessage(botResponse, 'bot');
            if (customerPhone) {
                saveMessageToHistory(customerPhone, botResponse, 'bot');
            }
            appendMenuMessage('Selecione outra opção:', menuOptions);
            setInputActive(true);
            userInput.focus();
            return;
        } else if (isSpecial && value === 'encerrar chat') {
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                if (customerPhone) {
                    saveMessageToHistory(customerPhone, data.resposta, 'bot');
                }
                setTimeout(() => {
                    toggleChatbot();
                    chatLog.innerHTML = '';
                    hasGreeted = false;
                    customerName = '';
                    customerPhone = '';
                    customerStreet = '';
                    customerNumber = '';
                    customerNeighborhood = '';
                    customerAddress = '';
                    currentInfoStep = 0;
                    collectingInfo = false;
                    setInputActive(false);
                }, 1500);
            }).catch(error => {
                console.error('Erro ao encerrar chat:', error);
                appendMessage('Desculpe, ocorreu um erro ao encerrar o chat.', 'bot');
                setInputActive(true);
            });
        } else if (value === 'pedido') {
            showConsolidatedOrderStatus();
            setInputActive(true);
        }
        else {
            fetch('/chat', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({ pergunta: value })
            })
            .then(res => res.json())
            .then(data => {
                appendMessage(data.resposta, 'bot');
                if (customerPhone) {
                    saveMessageToHistory(customerPhone, data.resposta, 'bot');
                }
                appendMenuMessage('Selecione outra opção:', menuOptions);
                setInputActive(true);
                userInput.focus();
            }).catch(error => {
                console.error('Erro ao processar opção do menu:', error);
                appendMessage('Desculpe, ocorreu um erro ao processar sua seleção.', 'bot');
                setInputActive(true);
            });
        }
    }

    function toggleChatbot() {
        console.log('toggleChatbot chamado. Display atual:', chatbotBox.style.display);
        if (chatbotBox.style.display === 'flex') {
            chatbotBox.style.display = 'none';
            console.log('Chatbot oculto.');
        } else {
            chatbotBox.style.display = 'flex';
            console.log('Chatbot visível. hasGreeted:', hasGreeted);
            if (!hasGreeted) {
                startCollectingInfo();
                hasGreeted = true;
            } else {
                setInputActive(true);
                if (!collectingInfo && currentInfoStep === 0) { 
                    appendMessage('Olá novamente! Como posso ajudar você?', 'bot');
                    appendMenuMessage('Selecione uma opção:', menuOptions);
                } else if (collectingInfo) {
                    if (currentInfoStep === 0) {
                        appendMessage('Ainda estou esperando seu número de telefone, por favor.', 'bot');
                    } else if (currentInfoStep === 1) {
                        appendMessage('Ainda estou esperando seu nome completo, por favor.', 'bot');
                    } else if (currentInfoStep === 2) {
                        appendMessage('Ainda estou esperando o nome da sua rua, por favor.', 'bot');
                    } else if (currentInfoStep === 3) {
                        appendMessage('Ainda estou esperando o número da sua casa ou apartamento, por favor.', 'bot');
                    } else if (currentInfoStep === 4) {
                        appendMessage('Ainda estou esperando o nome do seu bairro, por favor.', 'bot');
                    }
                }
                userInput.focus();
            }
        }
    }

    async function sendMessage() {
        const msg = userInput.value.trim();
        if (!msg) return;

        if (customerPhone) {
            await saveMessageToHistory(customerPhone, msg, 'user');
        }
        
        appendMessage(msg, 'user');
        userInput.value = '';

        if (collectingInfo) {
            await processCollectedInfo(msg);
            return;
        }
        
        let payload = { pergunta: msg }; 

        setInputActive(false);

        fetch('/chat', {
            method: 'POST',
            headers: {'Content-Type': 'application/json'},
            body: JSON.stringify(payload)
        })
        .then(res => {
            if (!res.ok) {
                throw new Error(`Erro de rede ou servidor: ${res.status}`);
            }
            return res.json();
        })
        .then(async data => {
            appendMessage(data.resposta, 'bot');
            if (customerPhone) {
                await saveMessageToHistory(customerPhone, data.resposta, 'bot');
            }
            
            if (payload.pergunta !== 'encerrar chat') {
                appendMenuMessage('Selecione outra opção:', menuOptions);
            }
            setInputActive(true);
            userInput.focus();
        })
        .catch(error => {
            console.error('Erro ao comunicar com o servidor:', error);
            appendMessage('Desculpe, ocorreu um erro ao processar sua solicitação. Por favor, tente novamente.', 'bot');
            setInputActive(true);
            userInput.focus();
        });
    }

    userInput.addEventListener('keydown', function(event) {
        if (event.key === 'Enter') {
            sendMessage();
            event.preventDefault();
        }
    });

    window.onload = function() {
        setInputActive(false);
    };
</script>

</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/get_user_data', methods=['POST'])
def get_user_data():
    data = request.json
    phone = data.get('phone', '')
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT name, street, number, neighborhood FROM users WHERE phone=?", (phone,))
    user = c.fetchone()
    conn.close()
    if user:
        return jsonify({
            'found': True,
            'name': user[0],
            'street': user[1],
            'number': user[2],
            'neighborhood': user[3]
        })
    return jsonify({'found': False})

@app.route('/save_user_data', methods=['POST'])
def save_user_data():
    data = request.json
    phone = data.get('phone', '')
    name = data.get('name', '')
    street = data.get('street', '')
    number = data.get('number', '')
    neighborhood = data.get('neighborhood', '')

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (phone, name, street, number, neighborhood) VALUES (?, ?, ?, ?, ?)",
                    (phone, name, street, number, neighborhood))
        conn.commit()
        return jsonify({'success': True})
    except sqlite3.IntegrityError:
        return jsonify({'success': False, 'message': 'Telefone já cadastrado.'})
    finally:
        conn.close()

@app.route('/save_chat_message', methods=['POST'])
def save_chat_message():
    data = request.json
    user_phone = data.get('phone', '')
    message_text = data.get('message_text', '')
    sender = data.get('sender', '')
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    try:
        c.execute("INSERT INTO chat_history (user_phone, message_text, sender, timestamp) VALUES (?, ?, ?, ?)",
                    (user_phone, message_text, sender, timestamp))
        conn.commit()
        return jsonify({'success': True})
    except Exception as e:
        print(f"Erro ao salvar mensagem: {e}")
        return jsonify({'success': False, 'message': str(e)})
    finally:
        conn.close()

@app.route('/get_chat_history', methods=['POST'])
def get_chat_history():
    data = request.json
    phone = data.get('phone', '')
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute("SELECT message_text, sender FROM chat_history WHERE user_phone=? ORDER BY timestamp", (phone,))
    history = [{'message_text': row[0], 'sender': row[1]} for row in c.fetchall()]
    conn.close()
    return jsonify({'history': history})

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    pergunta = data.get("pergunta", "").lower().strip()

    resposta = "Desculpe, não entendi sua pergunta. 🤔"
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()

    c.execute("SELECT resposta FROM chat WHERE LOWER(pergunta)=?", (pergunta,))
    row = c.fetchone()
    if row:
        resposta = row[0]

    conn.close()
    return jsonify({"resposta": resposta})

class ServerThread(Thread):
    def __init__(self, app):
        Thread.__init__(self)
        self.srv = make_server('0.0.0.0', 5001, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

server = ServerThread(app)
server.start()

display(HTML("<a href='http://localhost:5001' target='_blank'>🔗 Clique aqui para abrir o site com o chatbot</a>"))
